<a href="https://colab.research.google.com/github/penelopetir/24SBIOL74.03PT/blob/main/passive_membrane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting, we'll install neuron in our current runtime as usual.

In [80]:
%pip install neuron # only need to run this cell once to install
                    # neuron in the local jupyter environment

Run the code block below just once to get all the files from the repository into our colab session and compile the MOD mechanism files we'll be using

In [81]:
repo_name = 'BIOL74-Final-Project'
if 'google.colab' in str(get_ipython()):
    import os
    if not os.path.exists(repo_name):
        !git clone https://github.com/penelopetir/{repo_name}.git # downloads repository into our Google colab session's file system

    os.chdir(repo_name) # Changing working directory to downloaded repository

# Compile mechanisms
!nrnivmodl mechanisms

/content/BIOL74-Final-Project/BIOL74-Final-Project
Mod files: "mechanisms/mechanisms/CadepK.mod" "mechanisms/mechanisms/CaDynamics_E2.mod" "mechanisms/mechanisms/Ca_HVA.mod" "mechanisms/mechanisms/Ca_LVAst.mod" "mechanisms/mechanisms/Ca.mod" "mechanisms/mechanisms/Ih.mod" "mechanisms/mechanisms/Im.mod" "mechanisms/mechanisms/K_Pst.mod" "mechanisms/mechanisms/K_Tst.mod" "mechanisms/mechanisms/Nap_Et2.mod" "mechanisms/mechanisms/NaTa_t.mod" "mechanisms/mechanisms/ProbAMPANMDA.mod" "mechanisms/mechanisms/ProbGABAA.mod" "mechanisms/mechanisms/SK_E2.mod" "mechanisms/mechanisms/SKv3_1.mod"

 -> Compiling mod_func.cpp
 => LINKING shared library ./libnrnmech.so
Successfully created x86_64/special


## Introduction
For this exercise, we'll be adapting a model of a L5PC to model granule neurons traced from wild type and PTEN KO groups


Start by importing the packages we'll be using.

In [82]:
from neuron import h # all NEURON hoc functions are accessed through h
from neuron import gui # if you're running this notebook in a local environment (instead of on google colab), launches the GUI
h.load_file('stdrun.hoc') # loads neuron standard hoc library, not always necessary
import numpy as np
import matplotlib as mpl
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

if 'google.colab' in str(get_ipython()): # For making interactive plots work on google colab
    from google.colab import output
    output.enable_custom_widget_manager()
else:
    plotly.offline.init_notebook_mode() # for printing notebooks offline

#Importing Morphologies

Files from the github of traced neurons from NeuroLuicda 360 are imported and created with proper morphology. Biophysics are modeled after the Aradi et al 1999 paper.

In [95]:
from helper_functions import createGranuleCell

genotype = "KO" #change this string to "WT" or "KO" to determine which neuron to run the following code on

if genotype == "WT":
  WT_file = "morphologies/WT.asc" # morphology file
  GC = createGranuleCell(WT_file)
elif genotype == "KO":
  KO_file = "morphologies/PTEN_KO.asc" # morphology file
  GC = createGranuleCell(KO_file)
else:
  print("No neuron found")

# Create recording vectors for time and somatic voltage
t_vec = h.Vector().record(h._ref_t)
v_soma = h.Vector().record(GC.soma[0](0.5)._ref_v)

somastack_process 7
center
194.973	-263.9	-6	
Principle values
0.619399	37.2071	344.619	
Principle axes
 0.914    0.406    -0.0147 
 -0.402   0.9      -0.166  
 -0.054   0.157    0.986   
stk_triang_area = 781.997


In [96]:
#probing neurons for their parameters
print(f"{genotype} Soma Ra: {GC.soma[0].Ra} MOhm")
print(f"{genotype} Soma L: {GC.soma[0].L:.2f} um")
print(f"{genotype} Soma Diameter: {GC.soma[0].diam:.2f} um")
print(f"{genotype} Soma Cm: {GC.soma[0].cm} pF")
print(f"{genotype} Soma Area at point 0.5: {h.area(0.5)}")

KO Soma Ra: 210.0 MOhm
KO Soma L: 12.51 um
KO Soma Diameter: 18.46 um
KO Soma Cm: 1.0 pF
KO Soma Area at point 0.5: 748.3393903959693


#Measuring Rin and Cm
In this next section, we will be calibrating our model in an attempt to match the Williams et al 2015 paper results. The objective is to get our values of Rm and Cm to be as such:

WT:

KO:

The approach we will be taking is to apply a current stimulus and then measure the change in voltage of the soma membrane. With this, we can use Ohm's law to calculate the resistance by dividing the change in voltage by the current.

From here, we can take this value and then fit the decay time to a first-order exponential curve to calculate tau. From there, we can divide tau by Rin to get the Cm value.

In [85]:
#creating current clamp
iclamp = h.IClamp(GC.soma[0](0.5))

#generating a recording vector for current
Ic = h.Vector().record(iclamp._ref_i)

In [86]:
def run_and_plot_Iclamp_sim(t,v,ic):
    h.run()
    # Plot results
    fig = make_subplots(rows=2,cols=1)
    # voltage
    fig.add_trace(go.Scatter(x=t,y=v,showlegend=False),row=1,col=1)
    fig['layout']['yaxis']['title']='Vm (mV)'
    # Current
    fig.add_trace(go.Scatter(x=t,y=ic,showlegend=False),row=2,col=1)
    fig['layout']['xaxis2']['title']='time (ms)'
    fig['layout']['yaxis2']['title']='Ic (nA)'
    fig.show()

In [87]:
#running the current simulation
#setting parameters for the current clamp
iclamp.amp = -0.01 #nA, 3 pA
iclamp.dur = 500 #ms
iclamp.delay = 5 #ms

h.v_init = -70 # mV - initial voltage of the model system
h.dt = 0.1  # ms - time step
h.celsius = 6.3 # degrees celsius - temperature of simulation
h.tstop = 1000 # ms - simulation duration

run_and_plot_Iclamp_sim(t_vec, v_soma, Ic)

Changed dt


In [88]:
#helper function to find the index of the occurence of a value (num) in an array
def find_index(num, vector):
  diff = [np.abs(element - num) for element in np.array(vector)]
  return np.argmin(diff)

#function used to calculate the passive properties of the soma
def calc_passive(v_soma, amp):
  v_min = np.min(np.array(v_soma))
  v_max = -70 #mV

  Rin = np.abs((v_max - v_min)/(amp)) #mV/pA = MOhms

  v_soma_vec = np.array(v_soma)
  fall_start = find_index(v_min, v_soma_vec) #finds the index of the minimum voltage
  fall = v_soma_vec[fall_start:] #slices array so we only have the fall data
  t_init = t_vec[fall_start] #time at the minimum voltage, the start of the fall

  exp_decay = [0.63, 0.85, 0.96]
  tau = []
  for i, factor in enumerate(exp_decay):
    decay_to = v_min + factor*(v_max-v_min) #value to decay to from v_max
    t_val = t_vec[fall_start+find_index(decay_to, fall)]
    t_diff = t_val - t_init
    tau.append(t_diff/(i+1))

  ave_tau = np.mean(tau)
  cap = 1e3*ave_tau/Rin #to make it into pF

  return Rin, cap

In [89]:
Rin, cap = calc_passive(v_soma, iclamp.amp)
print(f"Rin = {Rin:.2f} MOhm")
print(f"Capacitance: {cap:.2f}pF")
tau = Rin*cap/1e3
print(f"Tau: {tau:.2f} ms")

Rin = 360.37 MOhm
Capacitance: 129.19pF
Tau: 46.56 ms


Now that we know our neuron's starting point, we need to try to get it to fit the model's capacitance and input resistances.

Control: 10pF, 600 M$\Omega$

PTEN KO: 40pF, 250 M$\Omega$